In [3]:
import os 
import sys
import numpy as np
import pandas as pd
import talib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler


In [4]:
df1 = pd.read_csv('/Users/zed/AI_Lab/DoubleEnsembleML/Data/BTC.csv')

In [10]:
df1 = df1.fillna(method='backfill')

In [15]:
df1 

,Date,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
0,2014-01-07,874.67040,892.06753,810.00000,810.00000,15.622378,1.315147e+04,841.835522
1,2014-01-08,810.00000,899.84281,788.00000,824.98287,19.182756,1.609733e+04,839.156269
2,2014-01-09,825.56345,870.00000,807.42084,841.86934,8.158335,6.784250e+03,831.572913
3,2014-01-10,839.99000,857.34056,817.00000,857.33056,8.024510,6.780220e+03,844.938794
4,2014-01-11,858.20000,918.05471,857.16554,899.84105,18.748285,1.669857e+04,890.671709
...,...,...,...,...,...,...,...,...
2652,2021-04-21,56500.00000,56805.50000,53600.00000,53815.20000,4788.013532,2.645352e+08,55249.477244
2653,2021-04-22,53815.30000,55459.40000,50500.00000,51730.00000,10144.204074,5.396125e+08,53194.167081
2654,2021-04-23,51709.30000,52124.40000,47549.30000,51178.70000,11159.243380,5.545896e+08,49697.773594
2655,2021-04-24,51178.60000,51200.00000,48746.40000,50093.40000,4039.459093,2.021033e+08,50032.268888


In [29]:
df1['r_0'] = df1.Close/df1.Close.shift(1)-1

In [22]:
def getDailyVol(close,span0 = 100):
    # df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    # df0 = df0[df0>0]
    # df0 = pd.Series(close.index[df0-1],index = close.index[close.shape[0]-df0.shape[0]])
    df0 = close/close.shift(1)-1
    df0 = df0.ewm(span = span0).std()
    return df0

In [27]:
dailyVol = getDailyVol(df1.Close)

In [46]:
df1['tag'] = pd.Series(map(lambda x,y: 1 if x>0.2*y else(-1 if x<-0.2*y else 0),df1.r_0,dailyVol))